In [343]:
import cython
import numpy as np
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [331]:
def _get_ones(x, y):
    return np.ones((x, y))


def _ext_A(A):
    nA, dim = A.shape
    A_ext = _get_ones(nA, dim * 3)
    A_ext[:, dim : 2 * dim] = A
    A_ext[:, 2 * dim :] = A ** 2
    return A_ext


def _ext_B(B):
    nB, dim = B.shape
    B_ext = _get_ones(dim * 3, nB)
    B_ext[:dim] = (B ** 2).T
    B_ext[dim : 2 * dim] = -2.0 * B.T
    del B
    return B_ext


def _euclidean(A_ext, B_ext):
    sqdist = A_ext.dot(B_ext).clip(min=0)
    return np.sqrt(sqdist)


def _norm(A):
    return A / np.linalg.norm(A, ord=2, axis=1, keepdims=True)


def _cosine(A_norm_ext, B_norm_ext):
    return A_norm_ext.dot(B_norm_ext).clip(min=0) / 2



In [349]:
n_samples = 100000
n_queries = 1
n_features = 10

query_vectors = np.random.random((n_queries,n_features))
X =np.random.random((n_samples,n_features))

In [350]:
def euclidean( _query_vectors, raw_B):
    data = _ext_B(raw_B)
    return _euclidean(_query_vectors, data)

def euclidean_vectorized(query_vectors, raw_B):
    _query_vectors = _ext_A(query_vectors)
    return euclidean( _query_vectors, raw_B)

In [353]:
%%timeit
euclidean_vectorized(query_vectors, X)

13.6 ms ± 308 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [355]:
def euclidean_naive(A,B):
    return np.sqrt(np.sum((A-B)**2,axis=1))

In [356]:
euclidean_naive(query_vectors, X).shape

(100000,)

In [357]:
euclidean_naive(query_vectors, X)

array([1.08393989, 1.17941507, 1.39265366, ..., 1.21415095, 1.03730183,
       1.40331934])

In [358]:
%%timeit 
euclidean_naive(query_vectors, X)

3.62 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [359]:
def cy_euclidean(q,X):
    n_samples = len(X)
    n_features = q.shape[1]
    
    result = np.zeros(len(X))
    for m in range(n_samples):
        res = 0.
        for i in range(n_features):
            res+= (q[0,i]- X[m, i])**2
        result[m] = np.sqrt(res)
    
    return result

In [360]:
cy_euclidean(query_vectors, X)

array([1.08393989, 1.17941507, 1.39265366, ..., 1.21415095, 1.03730183,
       1.40331934])

In [361]:
euclidean_naive(query_vectors, X)

array([1.08393989, 1.17941507, 1.39265366, ..., 1.21415095, 1.03730183,
       1.40331934])

In [362]:
%%cython

import numpy as np
import cython

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
def cy_euclidean(q,X):
    cdef int n_samples = len(X)
    cdef int n_features = len(q)
    cdef double res=0
    
    result = np.zeros(len(X))
    for m in range(n_samples):
        res = 0.
        for i in range(n_features):
            res+= (q[0,i]- X[m, i])**2
        result[m] = np.sqrt(res)
    
    return result

In [363]:
%%timeit
cy_euclidean(query_vectors, X)

162 ms ± 3.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [364]:
%%cython -a
import numpy as np
import cython 
from libc.math cimport pow, sqrt

@cython.boundscheck(False)  # Deactivate bounds checking
def cy_euclidean2(double[:,:] q,double[:,:] X):
    cdef int n_samples = X.shape[0]
    cdef int n_features = q.shape[1]
    cdef double res=0
    cdef double[:] result = np.zeros(len(X), dtype="double")

    for m in range(n_samples):
        res = 0.
        for i in range(n_features):
            res += pow(q[0,i]- X[m, i],2)
        result[m] = sqrt(res)
    
    return np.array(result)

In [365]:
cy_euclidean2(query_vectors, X)

array([1.08393989, 1.17941507, 1.39265366, ..., 1.21415095, 1.03730183,
       1.40331934])

In [366]:
euclidean_naive(query_vectors, X)

array([1.08393989, 1.17941507, 1.39265366, ..., 1.21415095, 1.03730183,
       1.40331934])

In [367]:
%%timeit
cy_euclidean2(query_vectors, X)

937 µs ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [368]:
%%timeit
euclidean_vectorized(query_vectors, X)

13.6 ms ± 197 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Todo: think vectorization

In [323]:
%%cython -a -c=-DUSE_XSIMD -c=-march=native

import numpy as np
cimport cython 
from cython.parallel import prange

from libc.math cimport pow, sqrt
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def cy_euclidean3(double[:,:] q,double[:,:] X):
    cdef int n_samples = X.shape[0]
    cdef int n_features = q.shape[1]
    cdef double partial = 0
    cdef double res=0
    cdef double[:] result = np.zeros(len(X), dtype="double")
    #cdef double[:] result = n_samples*[0]
    
    for m in range(n_samples):
        res = 0.
        
        for i in range(n_features):
            partial = q[0,i] - X[m, i]
            res = res + partial * partial
        result[m] = sqrt(res)
    
    return np.array(result)

In [312]:
cy_euclidean3(query_vectors, X)

array([3.1783345 , 2.46369614, 2.33293514, ..., 2.78822995, 2.97800318,
       2.31306567])

In [313]:
%%timeit
cy_euclidean3(query_vectors, X)

466 µs ± 33.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
